In [27]:
import pandas as pd
# Population data
pop_data = pd.read_excel('../population_data_UN.xlsx', index_col=0, header = 1)
pop_data = pop_data[["ISO3 Alpha-code",'Total Population, as of 1 January (thousands)']]
pop_data = pop_data.set_index("ISO3 Alpha-code")
country_codes = pd.read_csv('../countries_codes_and_coordinates.csv' , sep = ',', index_col = 0)

In [28]:
data = pd.read_csv('./results/optimization_results_world_grid.csv')
data = data.set_index('Country')
data = data.drop(['Antarctica'], axis =0)
for col in data.columns[1:]: 
    data[f'{col}_normalized']=data[col]/data['mean_load']
    
for c in data.index.to_list(): 
    results = pd.read_pickle(f'results/{c}/optimization_results.pickle')
    iso_code = country_codes.loc[c,'Alpha-3 code' ].split(' ')[1]
    pop = pop_data.loc[iso_code]
    pop=pop.iloc[0]
    data.loc[c, 'capacity_factor_wind'] = results['E_wind']/data.loc[c, 'P_wind']/8760
    data.loc[c, 'capacity_factor_pv'] = results['E_pv']/data.loc[c, 'P_pv']/8760
    data.loc[c, 'capacity_factor_dispatch'] = results['E_dispatch']/data.loc[c, 'P_dispatch']/8760
    data.loc[c, 'share_dispatchable'] = results['E_dispatch']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_wind']=results['E_wind']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'share_pv']=results['E_pv']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45/pop/1000
    data.loc[c, 'E_wind'] = results['E_wind']
    data.loc[c, 'E_pv'] = results['E_pv']
    data.loc[c, 'E_dispatch'] = results['E_dispatch']


In [40]:
data = data.drop_duplicates()

In [41]:
data_us = pd.read_csv('results/optimization_results_world_grid_with_states.csv')
data_us = data_us.set_index('State')

for col in data_us.columns[2:]: 
    data_us[f'{col}_normalized']=data_us[col]/data_us['mean_load']
# country_name = 'United States'
for i in range(len(data_us.index.to_list())): 
    c = data_us.index[i]
    country_name = data_us.iloc[i,0]
    results = pd.read_pickle(f'./results/{country_name}/optimization_results_{c}.pickle')
    iso_code = country_codes.loc[country_name,'Alpha-3 code' ].split(' ')[1]
    # pop = pop_data.loc[iso_code]
    # pop=pop.iloc[0]
    data_us.loc[c, 'capacity_factor_wind'] = results['E_wind']/data_us.loc[c, 'P_wind']/8760
    data_us.loc[c, 'capacity_factor_pv'] = results['E_pv']/data_us.loc[c, 'P_pv']/8760
    data_us.loc[c, 'capacity_factor_dispatch'] = results['E_dispatch']/data_us.loc[c, 'P_dispatch']/8760
    data_us.loc[c, 'share_dispatchable'] = results['E_dispatch']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data_us.loc[c, 'share_wind']=results['E_wind']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data_us.loc[c, 'share_pv']=results['E_pv']/(results['E_wind']+results['E_pv']+results['E_dispatch'])*100
    data_us.loc[c, 'E_wind'] = results['E_wind']
    data_us.loc[c, 'E_pv'] = results['E_pv']
    data_us.loc[c, 'E_dispatch'] = results['E_dispatch']
    # data_us.loc[c, 'carbon_impact']=results['E_wind']*11+results['E_pv']*45/pop/1000

In [42]:
data_to_merge = data.drop(['United States', 'China', 'Australia', 'Brazil', 'Russia', 'Canada'])
data_full =pd.concat([data_to_merge, data_us])
data_full = data_full.drop_duplicates()

In [43]:
data_full

,iso_alpha,mean_load,E_dispatch,P_dispatch,E_destock,P_pv,P_wind,mean_load_normalized,E_dispatch_normalized,P_dispatch_normalized,...,capacity_factor_wind,capacity_factor_pv,capacity_factor_dispatch,share_dispatchable,share_wind,share_pv,carbon_impact,E_wind,E_pv,Country
Afghanistan,AFG,810.466324,1.048195e+06,1147.380843,2.398239e+06,2615.638571,1544.225466,1.0,1293.323149,1.415705,...,0.107786,0.246219,0.104287,12.864633,17.895009,69.240357,7.311174e+08,1.458064e+06,5.641621e+06,NaN
Albania,ALB,833.338813,9.463228e+05,935.114592,1.980923e+06,3107.507896,1814.014819,1.0,1135.579919,1.122130,...,0.159912,0.174821,0.115524,11.475628,30.815032,57.709340,6.584861e+08,2.541122e+06,4.758926e+06,NaN
Algeria,DZA,7850.452511,9.212491e+06,11203.084726,4.783740e+06,12433.290247,11260.347368,1.0,1173.498120,1.427062,...,0.461972,0.213016,0.093872,11.813543,58.435190,29.751267,5.598792e+09,4.556920e+07,2.320077e+07,NaN
Angola,AGO,1114.159589,1.776980e+06,1436.186045,3.023659e+06,5198.508940,831.075936,1.0,1594.906452,1.289031,...,0.179414,0.185640,0.141243,15.402423,11.321585,73.275992,1.176663e+09,1.306173e+06,8.453865e+06,NaN
Argentina,ARG,17627.853425,1.375780e+07,21449.263214,1.153815e+07,21066.854165,37723.871678,1.0,780.457923,1.216782,...,0.361690,0.189089,0.073220,8.180508,71.070268,20.749224,8.938501e+09,1.195244e+08,3.489559e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CN043,CHN,16812.584475,2.567767e+07,20890.374521,4.302868e+07,82648.415688,24937.579778,1.0,1527.288547,1.242544,...,0.165812,0.153392,0.140315,14.846366,20.943002,64.210632,NaN,3.622216e+07,1.110561e+08,China
CN051,CHN,23142.097374,3.423075e+07,27157.000000,6.930323e+07,102688.744856,497895.831719,1.0,1479.155158,1.173489,...,0.007370,0.189629,0.143890,14.446066,13.565060,71.988874,NaN,3.214316e+07,1.705816e+08,China
CN045,CHN,15509.381050,1.894398e+07,17907.024399,4.142310e+07,69978.674370,15712.542932,1.0,1221.453183,1.154593,...,0.252147,0.165015,0.120766,12.237228,22.418988,65.343784,NaN,3.470597e+07,1.011562e+08,China
CN022,CHN,7572.393950,9.934583e+06,8713.104974,1.701307e+07,29005.738669,8895.026052,1.0,1311.947459,1.150641,...,0.319403,0.163116,0.130159,13.025758,32.631995,54.342247,NaN,2.488802e+07,4.144615e+07,China


In [44]:
for c in data_full.index.to_list(): 
    data_full.loc[c, 'carbon_impact']=data_full.loc[c,'E_wind']*11+data_full.loc[c,'E_pv']*45+data_full.loc[c,'E_dispatch']*440
for c in data.index.to_list(): 
    data.loc[c, 'carbon_impact']=data.loc[c,'E_wind']*11+data.loc[c,'E_pv']*45+data.loc[c,'E_dispatch']*440

In [48]:
data['mean_load'].sum()

2731083.6052511414

In [51]:
data_full['mean_load'].sum()

5396423.449315069

In [45]:
print("Résultats pour le monde désagréggé en états:")
print(f'Carbon impact : ' + str(data_full['carbon_impact'].sum()/10**12) + 'GtCO2') #Gt
print(f'E_pv : '+ str(data_full['E_pv'].sum()/10**6) +' TWh') #TWh
print(f'E_wind : '+str(data_full['E_wind'].sum()/10**6) +' TWh' )
print(f'P_dispatch : '+str(data_full['P_dispatch'].sum()/10**6)+' TW')
print(f'P_pv : ' + str(data_full['P_pv'].sum()/10**6)+' TW')
print(f'P_wind : ' +str(data_full['P_wind'].sum()/10**6) +' TW')

Résultats pour le monde désagréggé en états:
Carbon impact : 4.265687278083275GtCO2
E_pv : 23392.3373967671 TWh
E_wind : 19878.508280232898 TWh
P_dispatch : 7.44207246025097 TW
P_pv : 17.923230677385895 TW
P_wind : 10.49765175925934 TW


In [46]:
print("Résultats pour le monde désagréggé en pays:")
print(f'Carbon impact : ' + str(data['carbon_impact'].sum()/10**12) + 'GtCO2') #Gt
print(f'E_pv : '+ str(data['E_pv'].sum()/10**6) +' TWh') #TWh
print(f'E_wind : '+str(data['E_wind'].sum()/10**6) +' TWh' )
print(f'P_dispatch : '+str(data['P_dispatch'].sum()/10**6)+' TW')
print(f'P_pv : ' + str(data['P_pv'].sum()/10**6)+' TW')
print(f'P_wind : ' +str(data['P_wind'].sum()/10**6) +' TW')

Résultats pour le monde désagréggé en pays:
Carbon impact : 1.8771793055959711GtCO2
E_pv : 11301.28695060786 TWh
E_wind : 12623.00543139214 TWh
P_dispatch : 3.071806899034401 TW
P_pv : 7.532475748688562 TW
P_wind : 6.3402359133987405 TW


In [23]:
# Surface au sol
power_density_wind = 4 #W/m2 in Power Density
power_density_pv= 5.4 #W/m2

In [24]:
total_surface = (data['P_pv'].sum()/(power_density_pv*10**-6)+data['P_wind'].sum()/(power_density_wind*10**-6))
print(total_surface/10**6) #km2

3087860.16323923
